# Evaluating infrastructure adaptation options

This notebook forms the basis of "Hands-On 8" in the CCG course.

1. Take the risk results for the Ghana road damage and disruption analysis from previous hands-on sessions
2. Assume some adaptation options - explain what this means - and show their costs
3. Explain cost-benefit analysis (CBA) and show how to calculate Net Present Values for benefits (avoided risks) and costs

By the end of this tutorial you should be able to:
* Quantify the potential risk reduction of adaptation options
* Prioritise assets based on cost-benefit analysis for different adaptation options

In [9]:
# Imports from Python standard library
import math
import os
import warnings
from glob import glob
from pathlib import Path

# Imports from other Python packages
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

Change this to point to your data folder as in the previous tutorial:

In [10]:
data_folder = Path("../data")

## 1. Load risk results

Read in regions:

In [11]:
regions = gpd.read_file(
    data_folder
    / "gha_admbnda_gss_20210308_shp"
    / "gha_admbnda_gss_20210308_SHP"
    / "gha_admbnda_adm1_gss_20210308.shp"
)[["ADM1_PCODE", "ADM1_EN", "geometry"]]

Read in roads, join regions:

In [12]:
roads = gpd.read_file(
    data_folder / "GHA_OSM_roads.gpkg", layer="edges"
).rename(columns={"id": "road_id"})
roads = gpd.sjoin(roads, regions).drop(columns="index_right")
roads.head()

,osm_id,road_type,name,road_id,from_id,to_id,length_m,geometry,ADM1_PCODE,ADM1_EN
0,4790594,tertiary,Airport Road,roade_0,roadn_0,roadn_1,236.526837,"LINESTRING (-0.17544 5.60550, -0.17418 5.60555...",GH07,Greater Accra
1,4790599,tertiary,South Liberation Link,roade_1,roadn_2,roadn_10683,18.539418,"LINESTRING (-0.17889 5.59979, -0.17872 5.59977)",GH07,Greater Accra
2,4790599,tertiary,South Liberation Link,roade_2,roadn_10683,roadn_3,124.758045,"LINESTRING (-0.17872 5.59977, -0.17786 5.59960...",GH07,Greater Accra
3,4790600,tertiary,Airport Road,roade_3,roadn_4,roadn_6259,38.030821,"LINESTRING (-0.17330 5.60560, -0.17327 5.60556...",GH07,Greater Accra
4,4790600,tertiary,Airport Road,roade_4,roadn_6259,roadn_6258,19.532483,"LINESTRING (-0.17300 5.60559, -0.17299 5.60561...",GH07,Greater Accra


Read in risk:

In [44]:
risk = pd.read_csv(data_folder / "results" / "inunriver_damages_ead.csv")[
    ["id", "rcp", "gcm", "epoch", "ead_usd"]
].rename(columns={"id": "road_id"})
risk.head()

,road_id,rcp,gcm,epoch,ead_usd
0,roade_10012,historical,WATCH,1980,17873.887689
1,roade_10012,rcp4p5,GFDL-ESM2M,2030,22993.878433
2,roade_10012,rcp4p5,GFDL-ESM2M,2050,22993.878433
3,roade_10012,rcp4p5,GFDL-ESM2M,2080,22993.878433
4,roade_10012,rcp4p5,HadGEM2-ES,2030,22993.878433


In [16]:
exposed_roads = roads[roads.road_id.isin(risk.road_id.unique())]
exposed_roads

,osm_id,road_type,name,road_id,from_id,to_id,length_m,geometry,ADM1_PCODE,ADM1_EN
104,11154880,primary,La Road,roade_104,roadn_111,roadn_112,443.190787,"LINESTRING (-0.17564 5.55326, -0.17568 5.55324...",GH07,Greater Accra
126,11180537,trunk,Winneba Road,roade_126,roadn_135,roadn_9182,522.694931,"LINESTRING (-0.31338 5.55362, -0.31494 5.55356...",GH07,Greater Accra
127,11180537,trunk,Winneba Road,roade_127,roadn_9182,roadn_9181,54.297481,"LINESTRING (-0.31809 5.55347, -0.31858 5.55345)",GH07,Greater Accra
128,11180537,trunk,Winneba Road,roade_128,roadn_9181,roadn_9527,1075.851334,"LINESTRING (-0.31858 5.55345, -0.31866 5.55345...",GH07,Greater Accra
129,11180537,trunk,Winneba Road,roade_129,roadn_9527,roadn_402,185.212407,"LINESTRING (-0.32808 5.55182, -0.32844 5.55168...",GH07,Greater Accra
...,...,...,...,...,...,...,...,...,...,...
14304,863659491,trunk,Annor Assemah High Street,roade_14304,roadn_11547,roadn_11541,106.305695,"LINESTRING (-2.82481 5.82056, -2.82494 5.82032...",GH16,Western North
14305,863659492,trunk,Annor Assemah High Street,roade_14305,roadn_11542,roadn_11547,17.716419,"LINESTRING (-2.82473 5.82070, -2.82481 5.82056)",GH16,Western North
14368,903998624,tertiary,NaN,roade_14368,roadn_8938,roadn_11588,40.821448,"LINESTRING (-2.75989 5.85919, -2.76025 5.85911)",GH16,Western North
14369,903998625,tertiary,NaN,roade_14369,roadn_11588,roadn_9169,1836.249395,"LINESTRING (-2.76025 5.85911, -2.76162 5.85884...",GH16,Western North


In [41]:
exposure = pd.read_csv(data_folder / "results" / "inunriver_damages_rp.csv")[
    ["id", "length_m", "rcp", "gcm", "epoch", "rp"]
].rename(columns={"id": "road_id", "length_m": "flood_length_m"})

# sum over any segments exposed within the same return period
exposure = exposure.groupby(["road_id", "rcp", "gcm", "epoch", "rp"]).sum()

# # pick max length exposed over all return periods
exposure = (
    exposure.reset_index()
    .groupby(["road_id", "rcp", "gcm", "epoch"])
    .max()
    .reset_index()
)

exposure

,road_id,rcp,gcm,epoch,rp,flood_length_m
0,roade_10012,historical,WATCH,1980,1000,49.402745
1,roade_10012,rcp4p5,GFDL-ESM2M,2030,1000,49.402745
2,roade_10012,rcp4p5,GFDL-ESM2M,2050,1000,49.402745
3,roade_10012,rcp4p5,GFDL-ESM2M,2080,1000,49.402745
4,roade_10012,rcp4p5,HadGEM2-ES,2030,1000,49.402745
...,...,...,...,...,...,...
64810,roade_995,rcp8p5,MIROC-ESM-CHEM,2050,1000,1776.210973
64811,roade_995,rcp8p5,MIROC-ESM-CHEM,2080,1000,2708.082749
64812,roade_995,rcp8p5,NorESM1-M,2030,1000,727.427009
64813,roade_995,rcp8p5,NorESM1-M,2050,1000,727.427009


In [45]:
roads_with_risk = exposed_roads.merge(risk, on="road_id").merge(
    exposure, on=["road_id", "rcp", "gcm", "epoch"]
)
roads_with_risk.head(2)

,osm_id,road_type,name,road_id,from_id,to_id,length_m,geometry,ADM1_PCODE,ADM1_EN,rcp,gcm,epoch,ead_usd,rp,flood_length_m
0,11154880,primary,La Road,roade_104,roadn_111,roadn_112,443.190787,"LINESTRING (-0.17564 5.55326, -0.17568 5.55324...",GH07,Greater Accra,historical,WATCH,1980,2342.496724,1000,443.190787
1,11154880,primary,La Road,roade_104,roadn_111,roadn_112,443.190787,"LINESTRING (-0.17564 5.55326, -0.17568 5.55324...",GH07,Greater Accra,rcp4p5,GFDL-ESM2M,2030,2342.496724,1000,443.190787


## 2. Introduce adaptation options

Introduce costs of road upgrade options.

These costs are taken purely as an example, and further research is required to make reasonable estimates. They are intended represent upgrade to a bituminous or concrete road design, with a single-lane design for currently-unpaved roads. The routine maintenance costs are estimated for rehabilitation and routine maintenance that should take place every year. The periodic maintenance costs are estimated for resurfacing and surface treatment that may take place approximately every five years.

As before with cost estimates, the analysis is likely to be highly sensitive to these assumptions, which should be replaced by better estimates if available.

In [24]:
options = pd.DataFrame(
    {
        "kind": ["four_lane", "two_lane", "single_lane"],
        "initial_cost_usd_per_km": [1_000_000, 500_000, 125_000],
        "routine_usd_per_km": [20_000, 10_000, 5_000],
        "periodic_usd_per_km": [100_000, 50_000, 25_000],
    }
)
options

,kind,initial_cost_usd_per_km,routine_usd_per_km,periodic_usd_per_km
0,four_lane,1000000,20000,100000
1,two_lane,500000,10000,50000
2,single_lane,125000,5000,25000


Set a discount rate. This will be used to discount the cost of annual and periodic maintenance, as well as the present value of future expected annual damages.

This is another sensitive parameter which will affect the net present value calculations for both costs and benefits. As an exercise, try re-running the remainder of the analysis with different values here. What economic or financial justification could there be for assuming different discount rates?

In [25]:
discount_rate_percentage = 3

Given initial and routine costs and a discount rate, we can calculate the net present value for each adaptation option.

- start by calculating the normalised discount rate for each year over the time horizon
- add the initial costs for each option
- calculate the discounted routine costs for each option (assumed to be incurred each year)
- calculate the discounted periodic costs for each option (assumed to be incurred every five years)

In [26]:
# set up a costs dataframe
costs = pd.DataFrame()

# create a row per year over the time-horizon of interest
costs["year"] = np.arange(2020, 2081)
costs["year_from_start"] = costs.year - 2020

# calculate the normalised discount rate
discount_rate = 1 + discount_rate_percentage / 100
costs["discount_rate_norm"] = costs.year_from_start.apply(
    lambda y: 1.0 / math.pow(discount_rate, y)
)
# calculate the sum over normalised discount rates for the time horizon
# this will be useful later, to calculate NPV of expected damages
discount_rate_norm = costs.discount_rate_norm.sum()

# link each of the options, so we have a row per-option, per-year
costs["link"] = 1
options["link"] = 1
costs = costs.merge(options, on="link").drop(columns="link")

# set initial costs to zero in all years except start year
costs.loc[costs.year_from_start > 0, "initial_cost_usd_per_km"] = 0

# discount routine and periodic maintenance costs
costs.routine_usd_per_km = costs.discount_rate_norm * costs.routine_usd_per_km
costs.periodic_usd_per_km = (
    costs.discount_rate_norm * costs.periodic_usd_per_km
)
# set periodic costs to zero except for every five years
costs.loc[costs.year_from_start == 0, "periodic_usd_per_km"] = 0
costs.loc[costs.year_from_start % 5 != 0, "periodic_usd_per_km"] = 0
costs

,year,year_from_start,discount_rate_norm,kind,initial_cost_usd_per_km,routine_usd_per_km,periodic_usd_per_km
0,2020,0,1.000000,four_lane,1000000,20000.000000,0.000000
1,2020,0,1.000000,two_lane,500000,10000.000000,0.000000
2,2020,0,1.000000,single_lane,125000,5000.000000,0.000000
3,2021,1,0.970874,four_lane,0,19417.475728,0.000000
4,2021,1,0.970874,two_lane,0,9708.737864,0.000000
...,...,...,...,...,...,...,...
178,2079,59,0.174825,two_lane,0,1748.250827,0.000000
179,2079,59,0.174825,single_lane,0,874.125414,0.000000
180,2080,60,0.169733,four_lane,0,3394.661800,16973.309002
181,2080,60,0.169733,two_lane,0,1697.330900,8486.654501


This table can then be summarised by summing over all years in the time horizon, to calculate the net present value of all that future investment in maintenance.

In [27]:
npv_costs = (
    costs[
        [
            "kind",
            "initial_cost_usd_per_km",
            "routine_usd_per_km",
            "periodic_usd_per_km",
        ]
    ]
    .groupby("kind")
    .sum()
    .reset_index()
)
npv_costs["total_cost_usd_per_km"] = (
    npv_costs.initial_cost_usd_per_km
    + npv_costs.routine_usd_per_km
    + npv_costs.periodic_usd_per_km
)
npv_costs

,kind,initial_cost_usd_per_km,routine_usd_per_km,periodic_usd_per_km,total_cost_usd_per_km
0,four_lane,1000000,573511.273322,521281.893260,2.094793e+06
1,single_lane,125000,143377.818331,130320.473315,3.986983e+05
2,two_lane,500000,286755.636661,260640.946630,1.047397e+06


## 3. Estimate costs and benefits

Apply road kind assumptions for adaptation upgrades:

In [48]:
def kind(road_type):
    if road_type in ("trunk", "trunk_link", "motorway"):
        return "four_lane"
    elif road_type in ("primary", "primary_link", "secondary"):
        return "two_lane"
    else:
        return "single_lane"


roads_with_risk["kind"] = roads_with_risk.road_type.apply(kind)

Join adaptation cost estimates (per km)

In [49]:
roads_with_costs = roads_with_risk.merge(
    npv_costs[["kind", "total_cost_usd_per_km"]], on="kind"
)

Calculate total cost estimate for length of roads exposed

In [50]:
roads_with_costs["total_adaptation_cost_usd"] = (
    roads_with_costs.total_cost_usd_per_km
    / 1e3
    * roads_with_costs.flood_length_m
)

Calculate net present value of avoided damages over the time horizon:

In [51]:
roads_with_costs["total_adaptation_benefit_usd"] = (
    roads_with_costs.ead_usd * discount_rate_norm
)

In [52]:
discount_rate_norm

28.675563666119398

Calculate benefit-cost ratio

In [53]:
roads_with_costs["bcr"] = (
    roads_with_costs.total_adaptation_benefit_usd
    / roads_with_costs.total_adaptation_cost_usd
)

Filter to pull out just the historical climate scenario:

In [54]:
historical = roads_with_costs[roads_with_costs.rcp == "historical"]
historical.describe()

,length_m,epoch,ead_usd,rp,flood_length_m,total_cost_usd_per_km,total_adaptation_cost_usd,total_adaptation_benefit_usd,bcr
count,2264.000000,2264.0,2.264000e+03,2264.0,2264.000000,2.264000e+03,2.264000e+03,2.264000e+03,2264.000000
mean,3409.722027,1980.0,2.593277e+05,1000.0,1027.421512,1.049971e+06,8.582112e+05,7.436369e+06,7.745543
std,7251.226282,0.0,7.533146e+05,0.0,1959.983688,6.369807e+05,1.709022e+06,2.160172e+07,7.461552
min,1.290015,1980.0,0.000000e+00,1000.0,0.303766,3.986983e+05,1.211108e+02,0.000000e+00,0.000000
25%,47.555476,1980.0,3.338947e+03,1000.0,42.211808,3.986983e+05,4.214351e+04,9.574619e+04,0.635519
50%,366.251343,1980.0,1.977872e+04,1000.0,224.304937,1.047397e+06,1.980604e+05,5.671659e+05,9.905326
75%,3309.033698,1980.0,1.516975e+05,1000.0,970.686131,1.047397e+06,8.338113e+05,4.350010e+06,9.905326
max,73318.612176,1980.0,1.306064e+07,1000.0,17981.326559,2.094793e+06,1.856157e+07,3.745212e+08,20.177240


Filter to find cost-beneficial adaptation options under historic flood scenarios

In [55]:
candidates = historical[historical.bcr > 1]
candidates

,osm_id,road_type,name,road_id,from_id,to_id,length_m,geometry,ADM1_PCODE,ADM1_EN,...,gcm,epoch,ead_usd,rp,flood_length_m,kind,total_cost_usd_per_km,total_adaptation_cost_usd,total_adaptation_benefit_usd,bcr
31,11287763,primary,Obetsebi Lamptey Circle,roade_153,roadn_162,roadn_163,53.770003,"LINESTRING (-0.22956 5.56170, -0.22962 5.56166...",GH07,Greater Accra,...,WATCH,1980,19453.959302,1000,53.770003,two_lane,1.047397e+06,56318.517592,5.578532e+05,9.905326
62,11664722,primary,Ring Road Central,roade_181,roadn_194,roadn_195,115.414521,"LINESTRING (-0.21575 5.56968, -0.21578 5.56965...",GH07,Greater Accra,...,WATCH,1980,2287.927108,1000,6.323743,two_lane,1.047397e+06,6623.467290,6.560760e+04,9.905326
93,11665216,primary,Ring Road West,roade_183,roadn_198,roadn_199,528.631407,"LINESTRING (-0.22543 5.54138, -0.22519 5.54122...",GH07,Greater Accra,...,WATCH,1980,170494.756904,1000,471.241020,two_lane,1.047397e+06,493576.233867,4.889033e+06,9.905326
124,11665277,primary,Ring Road Central,roade_184,roadn_200,roadn_10659,305.440550,"LINESTRING (-0.22908 5.56154, -0.22904 5.56167...",GH07,Greater Accra,...,WATCH,1980,110508.232976,1000,305.440550,two_lane,1.047397e+06,319917.388862,3.168886e+06,9.905326
155,11665277,primary,Ring Road Central,roade_185,roadn_10659,roadn_201,23.379035,"LINESTRING (-0.22726 5.56358, -0.22718 5.56366...",GH07,Greater Accra,...,WATCH,1980,8458.522645,1000,23.379035,two_lane,1.047397e+06,24487.121053,2.425529e+05,9.905326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40958,634994013,trunk,NaN,roade_12169,roadn_10178,roadn_10180,64.086457,"LINESTRING (-2.82071 5.82178, -2.82097 5.82180...",GH16,Western North,...,WATCH,1980,94462.014008,1000,64.086457,four_lane,2.094793e+06,134247.872952,2.708751e+06,20.177240
40989,862409400,trunk,NaN,roade_14299,roadn_11541,roadn_5925,42.726103,"LINESTRING (-2.82515 5.81966, -2.82522 5.81928)",GH16,Western North,...,WATCH,1980,62977.325869,1000,42.726103,four_lane,2.094793e+06,89502.347911,1.805910e+06,20.177240
41020,862409401,trunk,Annor Assemah High Street,roade_14300,roadn_10179,roadn_11542,121.522488,"LINESTRING (-2.82397 5.82148, -2.82399 5.82146...",GH16,Western North,...,WATCH,1980,179121.446702,1000,121.522488,four_lane,2.094793e+06,254564.477292,5.136408e+06,20.177240
41063,863659491,trunk,Annor Assemah High Street,roade_14304,roadn_11547,roadn_11541,106.305695,"LINESTRING (-2.82481 5.82056, -2.82494 5.82032...",GH16,Western North,...,WATCH,1980,156692.232328,1000,106.305695,four_lane,2.094793e+06,222688.443806,4.493238e+06,20.177240


Summarise by region to explore where cost-beneficial adaptation options might be located.

We need to sum over exposed lengths of road, costs and benefits, while finding the mean benefit-cost ratio.

In [58]:
candidates.groupby("ADM1_EN").agg(
    {
        "flood_length_m": np.sum,
        "total_adaptation_benefit_usd": np.sum,
        "total_adaptation_cost_usd": np.sum,
        "bcr": np.mean,
    }
)

,flood_length_m,total_adaptation_benefit_usd,total_adaptation_cost_usd,bcr
ADM1_EN,,,,
Ahafo,6169.925987,5.334659e+07,6.462359e+06,9.201183
Ashanti,22377.092972,3.282192e+08,2.826401e+07,11.931011
Bono,7400.527071,5.550888e+07,7.751287e+06,9.460365
Bono East,27762.639348,4.182545e+08,3.792551e+07,8.978280
Central,315825.739173,5.869595e+09,4.171591e+08,15.218221
Eastern,95586.571989,1.185412e+09,1.208687e+08,11.219343
Greater Accra,144951.652415,2.633143e+09,1.993335e+08,12.986069
Northern,36063.690743,3.179162e+08,4.066786e+07,11.922071
Northern East,25863.284557,2.136089e+08,2.902658e+07,10.220424


Given the aggregation, filtering and plotting you've seen throughout these tutorials, what other statistics would be interesting to explore from these results?